In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
from sklearn.model_selection import train_test_split
import cv2

In [20]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train.astype('uint8')   
x_test = x_test.astype('uint8')

x_train.shape


x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.99)

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.3)

# x_train = x_train[:1000]
# x_test = x_test[:1000]
# y_train = y_train[:1000]
# y_test = y_test[:1000]

In [19]:
y_train.shape

(420,)

In [21]:
x_train = x_train.reshape(-1, 28*28)
x_test = x_test.reshape(-1, 28*28)
x_train = x_train.astype('float32')   
x_test = x_test.astype('float32')

In [22]:
kVals = [1,5,10]
accuracies = []
for k in kVals:
    # train the classifier with the current value of `k`
    model = KNeighborsClassifier(n_neighbors=k)

    model.fit(x_train, y_train)

    # evaluate the model and print the accuracies list
    score = model.score(x_test, y_test)
    print("k=%d, accuracy=%.2f%%" % (k, score * 100))
    accuracies.append(score)

k=1, accuracy=83.89%
k=5, accuracy=81.67%
k=10, accuracy=79.44%


In [23]:
kVals = [1,5,10]

for k in kVals:
    count = 0
    for i in range(len(x_test)):
        x = x_test[i]
        
        distances = [np.linalg.norm(x - data_point) for data_point in x_train]

        k_neighbors_indices = np.argsort(distances)[:k]
        k_neighbor_labels = [y_train[i] for i in k_neighbors_indices]
        
        # Return the most common class label
        most_common = np.bincount(k_neighbor_labels).argmax()
        if most_common == y_test[i]:
            count += 1
    score = count / len(x_test)
    print("k=%d, accuracy=%.2f%%" % (k, score * 100))



k=1, accuracy=83.89%
k=5, accuracy=81.67%
k=10, accuracy=79.44%


In [24]:
from sklearn.tree import DecisionTreeClassifier

In [25]:
# Train a Decision Tree on the data (Decision tree uses IG for the splits)
tree = DecisionTreeClassifier(random_state=0)
tree.fit(x_train, y_train)
importances = tree.feature_importances_
indices = np.argsort(importances)[::-1]
indices[0]

378

In [39]:
N = 100  # Change this to select number of features based on IG
selected_features = indices[:N]
print(selected_features)
# Apply the selection to training and test data
x_train_selected = x_train[:, selected_features]
x_test_selected = x_test[:, selected_features]

x_train[1][347]
# x_train_selected[1][0]

[378 542 402 656 625 346 461 359 211 155 269 345 457 522 271 153 326 405
 236 299 521 296 493 398 268 630 489 186 463 266 348 203 545 324 491 356
 602 379 265 651 343 458 682 377 177 638 432 494 179 157 401 513 215 462
 384 344 207 598 328 209 380 438 623 383 208 261 466 293 237 607 219 715
 331 187 425 320 546 414 436 255 258 257 256 235 254 253 252 259 262 260
 226 250 263 264 225 224 223 222 206 220]


208.0

In [27]:
kVals = [1, 5, 10, 20, 30]
accuracies = []

for k in kVals:
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(x_train_selected, y_train)
    score = model.score(x_test_selected, y_test)
    print(f"k={k}, accuracy={score * 100:.2f}%")
    accuracies.append(score)

k=1, accuracy=78.89%
k=5, accuracy=75.56%
k=10, accuracy=73.33%
k=20, accuracy=72.22%
k=30, accuracy=69.44%


In [28]:
kVals = [5,10,20,30,50,100]
for k in kVals:
    count = 0
    for i in range(len(x_test_selected)):
        x = x_test_selected[i]

        distance_matrix = np.argsort(abs(x - x_train_selected), axis=0)
        k_neighbor_labels = np.array([])
        k_star = k
        for l in range(len(x)):
            if k_star <= 0: break
            # print(l)
            # print('...')
            # print(distance_matrix[:, l][:k_star])
            distances_along_axis = np.array(distance_matrix[:,l][:k_star])

            k_neighbor_labels = np.concatenate([k_neighbor_labels,distances_along_axis])
            k_star -= 1


        # k_neighbors_indices = np.argsort(distances)[:k]
        # k_neighbor_labels = [y_train[i] for i in k_neighbors_indices]
        
        # Return the most common class label
        # print(k_neighbor_labels)
        
        labels = y_train[np.array(k_neighbor_labels).flatten().astype(int)]

        most_common = np.bincount(labels).argmax()
        if most_common == y_test[i]:
            count += 1
    score = count / len(x_test_selected)
    print("k=%d, accuracy=%.2f%%" % (k, score * 100))

k=5, accuracy=19.44%
k=10, accuracy=30.56%
k=20, accuracy=44.44%
k=30, accuracy=37.22%
k=50, accuracy=38.33%
k=100, accuracy=31.67%


In [44]:
covariance_matrix = np.cov(x_train, rowvar=False)
num_top_features = 20
top_features_indices = np.argsort(-np.abs(covariance_matrix), axis=1)[:, 1:num_top_features + 1]

kVals = [5,10,20,30,50,100, 120, 150, 180, 200]
for k in kVals:
    count = 0
    for i in range(len(x_test)):
        x = x_test[i]
        k_neighbor_labels = np.array([])
        k_star = k
        for l in selected_features:
            # distances = [np.linalg.norm(x - data_point) for data_point in x_train]
            # imp_feature_neighbours = top_features_indices[l] # THIS IS FOR COVARIANCE
            imp_feature_neighbours = np.array([l, l+1, l+27, l+28, l+29, l-1, l-27, l-28, l-29]) # this is for neighbour pixesl
            mask = imp_feature_neighbours <= 783
            imp_feature_neighbours = imp_feature_neighbours[mask]
            distances = np.linalg.norm(x[imp_feature_neighbours] - x_train[:,imp_feature_neighbours], axis=1)
            distances_along_axis = np.argsort(distances)[:k_star]
            k_neighbor_labels = np.concatenate([k_neighbor_labels,distances_along_axis])
            k_star = (k_star // 2) - 1
        
        labels = y_train[np.array(k_neighbor_labels).flatten().astype(int)]

        most_common = np.bincount(labels).argmax()
        if most_common == y_test[i]:
            count += 1
    score = count / len(x_test_selected)
    print("k=%d, accuracy=%.2f%%" % (k, score * 100))

k=5, accuracy=5.56%
k=10, accuracy=5.56%
k=20, accuracy=5.56%
k=30, accuracy=5.56%
k=50, accuracy=5.56%
k=100, accuracy=5.56%
k=120, accuracy=5.56%
k=150, accuracy=5.56%
k=180, accuracy=5.56%
k=200, accuracy=5.56%


Before k = 100, we are not even looking at all the features, it just bad. you should look at a good number of pixels if you want reasonable results, its not crazy for the values to be bad underneeth, how could you conclude without looking at that many pixels. So this is not actualy our 'k' its a slightly different hyperperameter

In [10]:
covariance_matrix = np.cov(x_train, rowvar=False)
num_top_features = 5
top_features_indices = np.argsort(-np.abs(covariance_matrix), axis=1)[:, 1:num_top_features + 1]

In [14]:
top_features_indices[372]

array([400, 345, 373, 344, 317], dtype=int64)

array([6731.7515, 4297.405 , 2842.059 , 6553.6353, 6194.0645],
      dtype=float32)